In [1]:
%run qsu.ipynb
import numpy as np
import sympy
import networkx as nx
from qecsim import paulitools as pt 
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode
import nbimporter
import galois
from typing import List, Tuple
import hypergraph_prod_code as hpc

In [2]:
def belief_prop(H: np.array, s: np.array, p: float, max_iter: int) -> Tuple:
    """ 
    Belief Propagation Algorithm for Decoding LDPC Codes

    Parameters:
    -----------
    H - parity-check matrix corresponding to either X or Z checks
    s - Error syndrome
    p - Channel error rate for chosen noise channel
    max_iter - Maximum number of iterations to run BP algorithm for
    """
    data_to_parity = np.zeros((len(H[0]),len(H)), dtype=float)
    parity_to_data = np.zeros((len(H), len(H[0])), dtype=float)
    H_tanner_graph = hpc.parity_check_mat_to_tanner(H)
    
    # Channel Log Likelihood Ratio
    p_l = np.log((1 - p)/p)
    
    P_1 = np.zeros((len(H[0]),), dtype=float)
    e_BP = np.zeros((len(H[0]),), dtype=float)

    # (1) Initialization
    for edge in H_tanner_graph.edges:
        data_node_num = int(edge[0][1:])
        parity_node_num = int(edge[1][1:])
        data_to_parity[data_node_num][parity_node_num] = p_l 

    for iter in range(1, max_iter + 1):
        # Scaling Factor
        a = 1 - 2**(-1 * iter)

        # (2) Parity to Data Messages
        for edge in H_tanner_graph.edges:
            parity_node_num = int(edge[1][1:])
            data_node_num = int(edge[0][1:])

            # Get list of neighbors of current parity_node set minus the current data node
            V = list(nx.neighbors(H_tanner_graph, edge[1]))
            V.remove(edge[0])

            # Get messages from elements of V to current parity node
            data_to_par_msgs = [data_to_parity[int(v[1:])][parity_node_num] for v in V]
            w = np.min([np.abs(msg) for msg in data_to_par_msgs])
            parity_to_data[parity_node_num][data_node_num] = ((-1) ** int(s[parity_node_num])) * a * np.prod(np.sign(data_to_par_msgs)) * w 

        # (3) Data to Parity Messages
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node set minus the current parity node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))
            U.remove(edge[1])

            # Get messages from elements of U to current data node
            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            data_to_parity[data_node_num][parity_node_num] = p_l + np.sum(par_to_data_msgs)

        # Hard Decision
        for edge in H_tanner_graph.edges:
            data_node_num = int(edge[0][1:])
            parity_node_num = int(edge[1][1:])

            # Get list of neighbors of current data node
            U = list(nx.neighbors(H_tanner_graph, edge[0]))

            par_to_data_msgs = [parity_to_data[int(u[1:])][data_node_num] for u in U]
            P_1[data_node_num] = p_l + np.sum(par_to_data_msgs)
            e_BP[data_node_num] = -1 * np.sign(P_1[data_node_num])
        
        # (4) Termination Check
        e_BP = e_BP * (e_BP > 0)
        print(e_BP)
        if (np.array_equal(np.dot(H, e_BP), s)):
            return True, e_BP, P_1 

    return False, e_BP, P_1

In [3]:
def mod(x,modulus):
    numer, denom = x.as_numer_denom()
    return numer*sympy.mod_inverse(denom,modulus) % modulus

# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0(H: np.array, P_1: np.array, s: np.array) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - parity check matrix
    P_1 - BP soft decision vector
    s - Error syndrome 

    Returns:
    --------
    Error string
    """
    GF = galois.GF(2)
    # Get the rank of the parity check matrix
    H_rank = np.linalg.matrix_rank(GF(H))
    print(H_rank)

    # Maintain a mapping between bit positions and elements of the BP soft-decision vector
    P_1_sorted_pos = np.argsort(P_1, kind='stable')[::-1]
    P_1_sorted = [P_1[i] for i in P_1_sorted_pos]
    print(P_1_sorted_pos)
    print(P_1_sorted)

    # Rearrange columns of H to match the reordered soft-decision vector
    H[:] = H[:, P_1_sorted_pos]

    # Select first RANK(H) linearly independent columns of above rearrangement
    H_rref, inds = sympy.Matrix(H).rref()
    #print(inds)

    H_rref = H_rref.applyfunc(lambda x : mod(x,2))
    H_rref, inds = sympy.Matrix(H_rref).rref()
    H_S = np.vstack(([H[:, inds[i]] for i in range(0, H_rank)]))
    
    H_rref, inds = sympy.Matrix(H).rref()
    H_rref = H_rref.applyfunc(lambda x : mod(x,2))
    H_rref, inds = sympy.Matrix(H_rref).rref()
    H_S = np.vstack(([H_S[:, inds[i]] for i in range(0, H_rank)])).T 
    H_S_inv = np.mod(np.linalg.inv(H_S), 2)
    

    # Calculate the OSD-0 solution on the basis-bits
    e_S = np.linalg.inv(H_S) @ s
    e_ST = np.hstack((e_S, np.zeros((len(H[0]) - H_rank,))))

    # Map the OSD-0 solution to the original bit-ordering
    e_OSD = np.zeros((len(H[0]),))
    for i in range(len(P_1_sorted_pos)):
        e_OSD[P_1_sorted_pos[i]] = e_ST[i]

    return e_OSD

In [37]:
def get_H_J(H:np.array, J: List, s_p: np.array):
    if (J == []):
        H_J = []
    else:
        H_J = np.vstack((H[:, i] for i in J)).T
    H_J_s = np.hstack((H_J,s_p))
    return H_J, H_J_s

def replace_pos(e_p: np.array, x: np.array, J: List):
    count = 0
    for i in range(len(e_p)):
        if(i in J):
            e_p[i] = x[count]
            count += 1

    return e_p


# Turn to Ordered Statistics Decoding if BP fails to converge
def OSD_0_V2(H: np.array, s: np.array, e_p: np.array, num_qubits: int) -> np.array:
    """ 
    The Ordered Statistics Decoding (OSD) Zero algorithm is a post-processing 
    algorithm utilized when BP fails to converge 

    Parameters:
    -----------
    H - Parity check matrix
    s - Error syndrome 
    e_p - Hard decision vector
    num_qubits - Number of qubits

    Returns:
    --------
    Error string e such that He = s
    """
    # Want to construct information set J
    J = []
    s_p = np.mod(s + (H @ e_p),2)
    for i in range(num_qubits):
        H_J, H_J_s = get_H_J(H, J, s_p)
        if (np.linalg.matrix_rank(H_J_s) == np.linalg.matrix_rank(H_J)):
            break 
        J_p = J + [i]
        H_J_p, _ = get_H_J(H, J_p, s_p)
        if (np.linalg.matrix_rank(H_J_p) > np.linalg.matrix_rank(H_J)):
            J = J_p
            s_p += e_p[i] * H[:,i]
    rref, _ = sympy.Matrix.rref(sympy.Matrix(H_J_s))
    x = rref[:,-1]
    e = replace_pos(e_p, x, s_p)
    return e

In [5]:
# initialize models
dim = 3
my_code = ToricCode(dim,dim)
my_error_model = DepolarizingErrorModel()
GF = galois.GF(2)

In [21]:
# Set physical error probability to 10%
error_probability = 0.1
# Seed random number generator for repeatability
#rng = np.random.default_rng(15355)
rng = np.random.default_rng()

# Error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
print(error)
print(('error:\n{}'.format(my_code.new_pauli(error))))

[0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0]
error:
┼─·─┼─·─┼─·
·   ·   ·  
┼─·─┼─·─┼─·
Z   ·   ·  
┼─·─┼─Y─┼─Y
·   Z   ·  


In [22]:
# Syndrome: Stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
print(syndrome)
print(('syndrome:\n{}'.format(my_code.ascii_art(syndrome))))

[0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0]
syndrome:
┼───X───┼──
│   │   │  
X───┼───┼──
│   │ Z │ Z
┼───┼───┼──
│   │ Z │ Z


In [23]:
Z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
X_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]
#print(str(Z_stabs) + "\n" + str(X_stabs))
#print(np.mod(X_stabs @ Z_stabs.T,2))
Z_error = error[2 * dim**2:]
X_error = error[:2 * dim**2]
print("Z_error: " + str(Z_error))
print("X_error: " + str(X_error))

Z_error: [0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0]
X_error: [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]


In [24]:
Z_syndrome = np.mod(X_stabs @ Z_error,2)
print("Z_syndrome: " + str(Z_syndrome))
X_syndrome = np.mod(Z_stabs @ X_error, 2)
print("X_syndrome: " + str(X_syndrome))

Z_syndrome: [1 0 0 0 0 0 0 1 0]
X_syndrome: [0 0 0 0 1 1 0 1 1]


In [25]:
converged_Z, e_BP_Z, P_1_Z = belief_prop(X_stabs, Z_syndrome, 0.1, 2 * dim ** 2)
converged_X, e_BP_X, P_1_X = belief_prop(Z_stabs, X_syndrome, 0.1, 2 * dim ** 2)

[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[ 1. -0. -0.  1. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -

In [26]:
print(converged_Z, e_BP_Z, P_1_Z)
print('\n')
print(converged_X, e_BP_X, P_1_X)

False [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.] [0.13227582 2.19722458 3.08871213 0.13227582 3.08871213 2.19722458
 3.08871213 5.15366089 5.15366089 0.13227582 0.13227582 3.08871213
 2.19722458 3.08871213 5.15366089 3.08871213 2.19722458 5.15366089]


False [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.] [6.59165697 2.19722458 2.19722458 6.59165697 2.19722458 2.19722458
 4.12830826 0.2661409  0.2661409  6.59165697 6.59165697 4.12830826
 2.19722458 2.19722458 0.2661409  2.19722458 2.19722458 0.2661409 ]


In [38]:
if (converged_Z == False):
    print("BP ON Z FAILED")
    e_OSD_Z = np.mod(OSD_0_V2(X_stabs, Z_syndrome, e_BP_Z, 2 * dim ** 2), 2)
    print(e_OSD_Z)
if (converged_X == False):
    print("BP ON X FAILED")
    e_OSD_X = np.mod(OSD_0_V2(Z_stabs, X_syndrome, e_BP_X, 2 * dim ** 2), 2)
    print(e_OSD_X)

BP ON Z FAILED


/tmp/ipykernel_4750/1624942106.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  H_J = np.vstack((H[:, i] for i in J)).T


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)